In [1]:
import pulp
import numpy as np
from pathlib import Path
from datetime import datetime
from collections import defaultdict

INPUT = Path("input.txt").read_text()

def extract_pair(pair):
    n, x = pair.strip().split()
    return x, int(n)

def parse(line):
    ingredients, result = line.strip().split(" => ")
    ingredients = [extract_pair(pair) for pair in ingredients.split(", ")]
    return ingredients, [extract_pair(result)]

def get_reactions(text):
    reactions = []
    for line in text.strip().splitlines():
        ingredients, result = parse(line)
        reaction = defaultdict(int)
        for x, n in ingredients:
            reaction[x] = -n
        for x, n in result:
            reaction[x] = n
        reactions.append(reaction)
    return reactions

### Part 1

In [2]:
def part1(text):
    reactions = get_reactions(text)
    products = {k for r in reactions for k in r.keys()}
    n = len(reactions)

    prob = pulp.LpProblem('part1', pulp.LpMinimize)
    variables = {}
    for i in range(n):
        variables[i] = pulp.LpVariable(f'reaction_{i:02d}', lowBound=0, cat='Integer')
        
    # decide how much ore we start with
    initial_ore = pulp.LpVariable(f'initial_ore', lowBound=0, cat='Integer')
    
    amounts = {}
    initial = {"ORE": initial_ore}
    for product in products:
        amounts[product] = sum(reactions[i].get(product, 0) * variables[i] for i in range(n)) + initial.get(product, 0)
        prob += (amounts[product] >= 0)
    
    # make at least one fuel
    prob += (amounts["FUEL"] >= 1)
    
    # objective function
    prob += initial_ore
    
    # solve
    optimization_result = prob.solve()
    assert optimization_result == pulp.LpStatusOptimal
    return int(initial_ore.value())

In [3]:
%%time
part1(INPUT)

CPU times: user 173 ms, sys: 134 ms, total: 308 ms
Wall time: 69.5 ms


612880

### Part 2

In [4]:
def part2(text):
    reactions = get_reactions(text)
    products = {k for r in reactions for k in r.keys()}
    n = len(reactions)

    prob = pulp.LpProblem('part2', pulp.LpMaximize)
    variables = {}
    for i in range(n):
        variables[i] = pulp.LpVariable(f'reaction_{i:02d}', lowBound=0, cat='Integer')
        
    amounts = {}
    initial = {"ORE": int(1e12)}
    for product in products:
        amounts[product] = sum(reactions[i].get(product, 0) * variables[i] for i in range(n)) + initial.get(product, 0)
        prob += (amounts[product] >= 0)
    
    # objective function
    prob += amounts["FUEL"]
    
    # solve
    optimization_result = prob.solve()
    assert optimization_result == pulp.LpStatusOptimal
    return int(amounts["FUEL"].value())

In [5]:
%%time
part2(INPUT)

CPU times: user 42 ms, sys: 8.81 ms, total: 50.8 ms
Wall time: 161 ms


2509120

### Tests

In [6]:
text = """
10 ORE => 10 A
1 ORE => 1 B
7 A, 1 B => 1 C
7 A, 1 C => 1 D
7 A, 1 D => 1 E
7 A, 1 E => 1 FUEL
"""
assert part1(text) == 31

In [7]:
text = """
157 ORE => 5 NZVS
165 ORE => 6 DCFZ
44 XJWVT, 5 KHKGT, 1 QDVJ, 29 NZVS, 9 GPVTF, 48 HKGWZ => 1 FUEL
12 HKGWZ, 1 GPVTF, 8 PSHF => 9 QDVJ
179 ORE => 7 PSHF
177 ORE => 5 HKGWZ
7 DCFZ, 7 PSHF => 2 XJWVT
165 ORE => 2 GPVTF
3 DCFZ, 7 NZVS, 5 HKGWZ, 10 PSHF => 8 KHKGT
"""
assert part1(text) == 13312
assert part2(text) == 82892753

In [8]:
text = """
2 VPVL, 7 FWMGM, 2 CXFTF, 11 MNCFX => 1 STKFG
17 NVRVD, 3 JNWZP => 8 VPVL
53 STKFG, 6 MNCFX, 46 VJHF, 81 HVMC, 68 CXFTF, 25 GNMV => 1 FUEL
22 VJHF, 37 MNCFX => 5 FWMGM
139 ORE => 4 NVRVD
144 ORE => 7 JNWZP
5 MNCFX, 7 RFSQX, 2 FWMGM, 2 VPVL, 19 CXFTF => 3 HVMC
5 VJHF, 7 MNCFX, 9 VPVL, 37 CXFTF => 6 GNMV
145 ORE => 6 MNCFX
1 NVRVD => 8 CXFTF
1 VJHF, 6 MNCFX => 4 RFSQX
176 ORE => 6 VJHF
"""
assert part1(text) == 180697
assert part2(text) == 5586022

In [9]:
text = """
171 ORE => 8 CNZTR
7 ZLQW, 3 BMBT, 9 XCVML, 26 XMNCP, 1 WPTQ, 2 MZWV, 1 RJRHP => 4 PLWSL
114 ORE => 4 BHXH
14 VRPVC => 6 BMBT
6 BHXH, 18 KTJDG, 12 WPTQ, 7 PLWSL, 31 FHTLT, 37 ZDVW => 1 FUEL
6 WPTQ, 2 BMBT, 8 ZLQW, 18 KTJDG, 1 XMNCP, 6 MZWV, 1 RJRHP => 6 FHTLT
15 XDBXC, 2 LTCX, 1 VRPVC => 6 ZLQW
13 WPTQ, 10 LTCX, 3 RJRHP, 14 XMNCP, 2 MZWV, 1 ZLQW => 1 ZDVW
5 BMBT => 4 WPTQ
189 ORE => 9 KTJDG
1 MZWV, 17 XDBXC, 3 XCVML => 2 XMNCP
12 VRPVC, 27 CNZTR => 2 XDBXC
15 KTJDG, 12 BHXH => 5 XCVML
3 BHXH, 2 VRPVC => 7 MZWV
121 ORE => 7 VRPVC
7 XCVML => 6 RJRHP
5 BHXH, 4 VRPVC => 5 LTCX
"""
assert part1(text) == 2210736
assert part2(text) == 460664